In [1]:
import cv2
import numpy as np


capture = cv2.VideoCapture(0)

ret, frame1 = capture.read()

face_cascade = cv2.CascadeClassifier('../../notebooks/DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame1)

face_x, face_y, width, height = tuple(face_rects[0])
track_window = (face_x, face_y, width, height)

roi = frame1[face_y:face_y + height, face_x:face_x + width]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

roi_histogram = cv2.calcHist([hsv_roi], channels=[0], mask=None, histSize=[180], ranges=[0, 180])

cv2.normalize(roi_histogram, roi_histogram, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while capture.isOpened():

    ret, frame = capture.read()

    if ret:

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # Calculate back projection base off 'roi_histogram'
        dst = cv2.calcBackProject(hsv, channels=[0], hist=roi_histogram, ranges=[0, 180], scale=1)
        ################### Calculate the MeanShift and Draw Rectangle ########################
        # ret, track_windowd = cv2.meanShift(probImage=dst, window=track_window, criteria=criteria)
        #
        # x, y, w, h = track_window
        # img2 = cv2.rectangle(frame,
        #                      pt1=(x, y),
        #                      pt2=(x + w, y + h),
        #                      color=(0, 0, 255),
        #                      thickness=5)
        #######################################################################################

        ################### Calculate the CamShift and Draw Rectangle #########################
        ret, track_window = cv2.CamShift(probImage=dst, window=track_window, criteria=criteria)

        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, pts=[pts], isClosed=True, color=(0, 0, 255), thickness=5)
        #######################################################################################

        cv2.imshow('Tracking', img2)

        if cv2.waitKey(20) & 0xff == 27:
            break
    else:
        break

cv2.destroyWindow('Tracking')
capture.release()